In [ ]:
#Requirement already satisfied #in 2020 11 01
#!pip install pip install tensorflow-addons

!pwd

!nvidia-smi

In [ ]:
#https://www.kaggle.com/yasufuminakama/moa-pytorch-nn-starter
#part of codes are derived from above note by Y.Nakama

import tensorflow.keras.layers


import numpy as np
import pandas as pd




train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
val_features = pd.read_csv('../input/lish-moa/test_features.csv')
submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [ ]:
# ref: https://www.kaggle.com/c/lish-moa/discussion/180165
# check if labels for 'ctl_vehicle' are all 0.
train = train_features.merge(train_targets_scored, on='sig_id')
print(train.shape)

In [ ]:

target_cols = []

for c in train_targets_scored.columns:
    if c != 'sig_id':
        target_cols.append(c)
print(len(target_cols))
print(target_cols)

cols = target_cols + ["cp_type"]
#print(cols)

In [ ]:
# constrcut train&test except 'cp_type'=='ctl_vehicle' data
print(train_features.shape, val_features.shape)
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
val = val_features[val_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)
print(train.shape, val.shape)

In [ ]:
def cate2num(df):
    df['cp_time'] = df['cp_time'].map({24: 0, 48: 1, 72: 2})
    df['cp_dose'] = df['cp_dose'].map({'D1': 3, 'D2': 4})
    return df

train = cate2num(train)
val = cate2num(val)



In [ ]:
print(train.shape,val.shape)

In [ ]:
train_input=train[train_features.columns].values[:,2:].astype(np.float32)
print(train_input.shape )
print(train_input.dtype)
print(train_input)

# split the input into g and c gained the score little

In [ ]:
g_num=770
train_input_g = train_input[:,:g_num]
train_input_c = train_input[:,g_num:]
print(train_input_g.shape)
print(train_input_c.shape)

In [ ]:
train_label=train[target_cols].values.astype(np.float32)
print(train_label.shape)

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dropout, Conv2D, BatchNormalization, Activation,Multiply,Dense,Add
from tensorflow.keras.layers import UpSampling2D, add, concatenate, MaxPooling2D, AveragePooling2D,GlobalAveragePooling2D, Concatenate
from tensorflow_addons.optimizers import RectifiedAdam

def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred,0.001,0.999)
    return -K.mean(y_true*K.log(y_pred) + (1-y_true)*K.log(1-y_pred))


input_g = Input(shape=(train_input_g.shape[-1],))

x = Dense(2048)(input_g)
x = BatchNormalization()(x)
x = Activation(tf.nn.swish)(x)

x = Dense(1024)(x)
x = BatchNormalization()(x)
out_g = Activation(tf.nn.swish)(x)

input_c = Input(shape=(train_input_c.shape[-1],))
x = Dense(2048)(input_c)
x = BatchNormalization()(x)
x = Activation(tf.nn.swish)(x)

x = Dense(1024)(x)
x = BatchNormalization()(x)
out_c = Activation(tf.nn.swish)(x)

concat = Concatenate()([out_g,out_c])

x = Dropout(0.2)(concat)

out = Dense(train_label.shape[-1],activation="sigmoid")(x)

model = Model(inputs=[input_g,input_c], outputs=out)

model.summary()

In [ ]:
tf.keras.utils.plot_model(model, to_file="model.png", show_shapes=True)

In [ ]:
#opt = tf.keras.optimizers.SGD(0.01)
#opt = tf.keras.optimizers.Adam(0.001)
opt = RectifiedAdam(0.0001)
#
#model.compile(optimizer=opt,loss="binary_crossentropy",metrics =["accuracy"])
#model.compile(optimizer=opt,loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.001)
#              ,metrics =[logloss])

model.compile(optimizer=opt,loss=tf.keras.losses.Huber(delta=1.0, name='huber_loss')
              ,metrics =[logloss])

epochs = 1000
result = model.fit(x= [train_input_g,train_input_c], y = train_label, epochs= epochs, batch_size=2048, validation_split=0.8, verbose=0)

In [ ]:
print("last_val_logloss",result.history["val_logloss"][-1])
print("best_val_logloss",min(result.history["val_logloss"]))
print("best_val_logloss_epoch",np.argmin(np.array(result.history["val_logloss"])))
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(range(1, epochs+1), result.history["loss"], label="training")
plt.plot(range(1, epochs+1), result.history["val_loss"], label="validation")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()


In [ ]:
pred = model.predict([train_input_g[0:20],train_input_c[0:20]])
model.evaluate([train_input_g[0:20],train_input_c[0:20]],train_label[0:20])


In [ ]:
pred_idx = np.argmax(pred,axis=1)
print(pred_idx)

In [ ]:

for i in range(20):
    tmp = train_label[i]
    print(np.argmax(tmp),pred_idx[i])

In [ ]:
val_input=val[val.columns].values[:,2:].astype(np.float32)
print(val_input.shape )
print(val_input.dtype)
print(val_input)

In [ ]:
print(val_input.shape)
val_input_g = val_input[:,:g_num]
val_input_c = val_input[:,g_num:]
print(val_input_g.shape,val_input_c.shape)

In [ ]:
predictions = model.predict([val_input_g,val_input_c])

In [ ]:
print(predictions.shape)
print(predictions[0:10])

In [ ]:
#pred_sub = np.where(predictions>0.5,0.999,0.001)
#pred_sub = np.where(predictions<0.001,0.001,predictions)
#clipping
pred_sub = np.clip(predictions,0.001,0.999)

In [ ]:
#print(target_cols)
val[target_cols] = pred_sub
#val[["sig_id"]+target_cols].to_csv("./pred.csv",index=False)

In [ ]:
sub = submission.drop(columns=target_cols).merge(val[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub.to_csv('./submission.csv', index=False)
sub.head()